In [3]:

import re  # 表示
from jsonpath import jsonpath  # 解析json数据
import requests
import pandas as pd  # 存取csv数据
import datetime  # 转换时间
from datetime import datetime as dt
import csv
import time
import random
# datetime.now()
import os
import csv


def trans_time(v_str):
    GMT_FORMAT = '%a %b %d %H:%M:%S +0800 %Y'
    timeArray = datetime.datetime.strptime(v_str, GMT_FORMAT)
    timeArray = dt.strptime(v_str, GMT_FORMAT)
    ret_time = timeArray.strftime("%Y-%m-%d %H:%M:%S")
    return ret_time


def get_weibo_list(v_keyword, v_max_page):
    headers = {
        "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
        "accept": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        "accept-encoding": "gzip,deflate,br"
    }
    for page in range(1, v_max_page + 1):
        #        print("===开始爬取第{}页微博===".format(page))
        url = "https://m.weibo.cn/api/container/getIndex"
        params = {
            "containerid": "100103type=1&q={}".format(v_keyword),
            "page_type": "searchall",
            "page": page
        }
        # 发送请求
        r = requests.get(url, headers=headers, params=params)
        print(r.status_code)
        # pprint(r.json())
        # 解析json数据
        cards = r.json()["data"]["cards"]
        # 微博内容
        text_list = jsonpath(cards, "$..mblog.text")
        # 微博内容-正则表达式数据清洗
        dr = re.compile(r"<[^>]+>", re.S)
        text2_list = []
        print("text_list is:")
        print(text_list)
        if not text_list:
            continue
        if type(text_list) == list and len(text_list) > 0:
            for text in text_list:
                text2 = dr.sub('', text)
                # print(text2)
                text2_list.append(text2)
        # 微博创建时间
        time_list = jsonpath(cards, "$..mblog.created_at")
        time_list = [trans_time(v_str=i) for i in time_list]
        # 微博作者
        author_list = jsonpath(cards, "$..mblog.user.screen_name")
        # 微博id
        id_list = jsonpath(cards, "$..mblog.id")
        # 微博bid
        bid_list = jsonpath(cards, "$..mblog.bid")
        # 转发数
        reposts_count_list = jsonpath(cards, "$..mblog.reposts_count")
        # 评论数
        comments_count_list = jsonpath(cards, "$..mblog.comments_count")
        # 点赞数
        attitudes_count_list = jsonpath(cards, "$..mblog.attitudes_count")
        # 把数据保存为dataframe

        for allinfo in zip(time_list, text2_list, author_list, id_list, bid_list, reposts_count_list,
                           comments_count_list, attitudes_count_list):
            print(allinfo)
            with open('data.csv', 'a', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(allinfo)

        time.sleep(random.uniform(0.5, 2))


if __name__ == '__main__':
    max_search_page = 10
    search_keyword = "飞天奖"
    v_weibo_file = "微博清单_{}_前{}页.csv".format(search_keyword, max_search_page)
    # 如果csv文件存在，需要先删除
    if os.path.exists(v_weibo_file):
        os.remove(v_weibo_file)
        print("微博清单存在，已删除:{}".format(v_weibo_file))
    # 调用爬取数据
    get_weibo_list(v_keyword=search_keyword, v_max_page=max_search_page)

200
text_list is:
['<a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E9%A3%9E%E5%A4%A9%E5%A5%96%E9%A2%81%E5%A5%96%E7%A4%BC%E5%87%BA%E5%B8%AD%E5%90%8D%E5%8D%95%23&extparam=%23%E9%A3%9E%E5%A4%A9%E5%A5%96%E9%A2%81%E5%A5%96%E7%A4%BC%E5%87%BA%E5%B8%AD%E5%90%8D%E5%8D%95%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E9%A3%9E%E5%A4%A9%E5%A5%96" data-hide=""><span class="surl-text">#飞天奖颁奖礼出席名单#</span></a><a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E6%9C%B1%E4%B8%80%E9%BE%99%E5%B0%86%E5%87%BA%E5%B8%AD%E9%A3%9E%E5%A4%A9%E5%A5%96%E9%A2%81%E5%A5%96%E7%A4%BC%23&extparam=%23%E6%9C%B1%E4%B8%80%E9%BE%99%E5%B0%86%E5%87%BA%E5%B8%AD%E9%A3%9E%E5%A4%A9%E5%A5%96%E9%A2%81%E5%A5%96%E7%A4%BC%23&luicode=10000011&lfid=100103type%3D1%26q%3D%E9%A3%9E%E5%A4%A9%E5%A5%96" data-hide=""><span class="surl-text">#朱一龙将出席飞天奖颁奖礼#</span></a> 在飞天奖发布会上，总导演透露飞天奖颁奖礼情况，而飞天奖颁奖礼出席名单也随之曝光！飞天奖颁奖礼分为三个板块，第一版块31个卫视主持人共同完成“春天是我们的”；第二版块陈宝国、张嘉译、何冰、黄轩、孙俪、周迅等等来讲述“荣誉是人民的”；